In [1]:
import sys
import os

thesis_path = "/" + os.path.join(
    *os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [2]:
from evaluations.content_02 import get_per_sample_metrics

metrics_df = get_per_sample_metrics()

100%|██████████| 2160/2160 [02:40<00:00, 13.46it/s]


In [4]:
metrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130766400 entries, 0 to 130766399
Data columns (total 18 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   seed            int64 
 1   checkpoint      object
 2   gazetteer_size  int64 
 3   error_ratio     int64 
 4   error_part      object
 5   timestep        int64 
 6   dataset         object
 7   doc_id          object
 8   targets         int64 
 9   entity_type     object
 10  tp              int64 
 11  fn              int64 
 12  fp              int64 
 13  error_type1     int64 
 14  error_type2     int64 
 15  error_type3     int64 
 16  error_type4     int64 
 17  error_type5     int64 
dtypes: int64(13), object(5)
memory usage: 17.5+ GB
